In [168]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import re 

# Example 4. PMF (Passenger Mix Flow) model

In [169]:
# Define the inputs

# L: set of flights
flights = pd.read_excel('Group_4_P2_PMF.xlsx', sheet_name='Flight').set_index('Flight Number')

# P: set of passenger itineraries
paths = pd.read_excel('Group_4_P2_PMF.xlsx', sheet_name='Itinerary').set_index('Itin No.')
paths['Leg 2'] = paths['Leg 2'].astype('Int64')

# P_p: set of passenger itineraries with recapture rate from itinerary p
P_p = pd.read_excel('Group_4_P2_PMF.xlsx', sheet_name='Recapture Rate').set_index(['p','r'])

# make a dictionary with the itinerary as the key and the rest as a sub-dictionary
paths = paths.to_dict(orient='index')
flights = flights.to_dict(orient='index')
P_p = P_p.to_dict(orient='index')

path_list = list(paths.keys())
flight_list = list(flights.keys())

# For all paths if 'Leg 1' and 'Leg 2' are numbers then create a list with both legs else, drop the keys from the list, and create a new key called 'Legs'
# else just change the name of the key 'Leg 1' to 'Legs'
for key in path_list:
    legs = []
    if type(paths[key]['Leg 1']) == int and type(paths[key]['Leg 2']) == int:
        legs.append(paths[key]['Leg 1'])
        legs.append(paths[key]['Leg 2'])
        paths[key]['Legs'] = legs
    elif type(paths[key]['Leg 1']) == int:
        legs.append(paths[key]['Leg 1'])
        paths[key]['Legs'] = legs
    del paths[key]['Leg 1']
    del paths[key]['Leg 2']

# Define path 999 with a fare of 0 and a demand of 0 
paths[999] = {'Legs': [], 'Demand': 0, 'Fare': 0}

In [170]:
# s_ip: binary variable indicating whether flight i is in itinerary p
s_ip = {}
for i in flight_list:
    for p in paths:
        s_ip[i,999] = 0
        if i in paths[p]['Legs']:
            s_ip[i,p] = 1
        else:
            s_ip[i,p] = 0

# Q_i: unconstrained demand for flight i = sum s_ip * demand of itinerary p for p in P
Q_i = {}
for i in flight_list:
    Q_i[i] = 0
    for p in paths:
        Q_i[i] += s_ip[i,p] * paths[p]['Demand']

# ds_i demand spill for flight i = Q_i - capacity of flight i
ds_i = {}
for i in flight_list:
    ds_i[i] = Q_i[i] - flights[i]['Capacity']

In [171]:
# Add entries to P_p for path 0 with a recapture rate of 1
for p in paths:
    P_p[p,999] = {'Recapture rate': 1}
    P_p[999,p] = {'Recapture rate': 0}

path_list = list(paths.keys())
flight_list = list(flights.keys())

In [172]:
initial = [999]

In [173]:
# Implementation using column generation algorithm (CGA)

# Define the restricted master problem (RMP)
# All the spillage is reallocated to path 999

# Define the model
m = gp.Model('PMF')

# Define the decision variables
# t_pr: number of passengers that would like to fly on itinerary p and are reallocated to itinerary r
t = {}
for p in path_list:
    for r in initial:
        t[p,r] = m.addVar(name='t_'+str(p)+'_'+str(r), lb=0)

Give me an example of a Itinerary-based Fleet Assignment Problem where column generation is used to reduce the computational needs. But in order to create such problem a procing problem needs to be solved every iteration. i am interested in getting from you how to create the pricing problem

In [174]:
# Define the objective function
# MINIMIZE (double sum of fare_p - bpr * fare_r) * t_pr
of = gp.quicksum((paths[p]['Fare'] - P_p[(p,r)]['Recapture rate'] * paths[r]['Fare']) * t[p,r] for r in initial for p in path_list)
m.setObjective(of, GRB.MINIMIZE)

# Define the constraints
# Constraint 1: sum sum s_ip * t_pr - sum sum s_ip * brp * t_rp >= ds_i for all i but for r = 0 
m.addConstrs((gp.quicksum(s_ip[i,p] * t[p,r] for p in path_list for r in initial) - 
              gp.quicksum(s_ip[i,p] * P_p[(r,p)]['Recapture rate'] * t[p,r] for p in path_list for r in initial) >= 
              ds_i[i] for i in flight_list), name='π')

# Constraint 2: sum t_pr <= Dp for all p
for p in path_list:
    m.addConstr((gp.quicksumt([p,r] <= paths[p]['Demand'] for r in initial)), name='σ')

# Update the model
m.update()

# Solve the model
m.optimize()


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1308 rows, 550 columns and 1990 nonzeros
Model fingerprint: 0x86bbed67
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 1287 rows and 431 columns
Presolve time: 0.01s
Presolved: 21 rows, 119 columns, 129 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.1035000e+04   1.741250e+02   0.000000e+00      0s
      26    1.3029500e+05   0.000000e+00   0.000000e+00      0s

Solved in 26 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.302950000e+05


In [175]:
# Print the optimal objective value and the decision variables t_pr and the dual variables
print('Optimal objective value: %g' % m.objVal)
print('Optimal solution:')
for v in m.getVars():
    if v.x > 0:
        print('%s = %g' % (v.varName, v.x))

print('Dual variables:')
for c in m.getConstrs():
    if c.Pi != 0:
        print('%s = %g' % (c.ConstrName, c.Pi))

# Save dual variables in a dictionary
pi_dual = {}
for c in m.getConstrs():
    if c.constrName[0] == 'π':
        # get only the flight number from the constraint name
        flight_num_pi = int(re.findall(r'\d+', c.ConstrName)[0])    
        pi_dual[flight_num_pi] = c.Pi

sigma_dual = {}
for c in m.getConstrs():
    if c.constrName[0] == 'σ':
        path_num_sigma = int(re.findall(r'\d+', c.ConstrName)[0])    
        sigma_dual[path_num_sigma] = c.Pi
        

Optimal objective value: 130295
Optimal solution:
t_9_999 = 5
t_19_999 = 26
t_48_999 = 64
t_65_999 = 19
t_69_999 = 1
t_80_999 = 14
t_112_999 = 24
t_118_999 = 10
t_119_999 = 8
t_124_999 = 23
t_125_999 = 20
t_126_999 = 15
t_127_999 = 12
t_128_999 = 10
t_129_999 = 30
t_130_999 = 19
t_203_999 = 37
t_227_999 = 53
t_228_999 = 12
t_230_999 = 9
t_231_999 = 24
t_232_999 = 18
t_233_999 = 5
t_241_999 = 12
t_266_999 = 28
t_269_999 = 4
t_271_999 = 22
t_300_999 = 28
t_301_999 = 24
t_302_999 = 12
t_304_999 = 19
t_312_999 = 34
t_325_999 = 18
t_334_999 = 27
t_341_999 = 8
t_342_999 = 5
t_345_999 = 10
t_346_999 = 43
t_347_999 = 14
t_350_999 = 60
t_357_999 = 19
t_375_999 = 37
t_380_999 = 9
t_382_999 = 20
t_386_999 = 11
t_390_999 = 19
t_393_999 = 26
t_409_999 = 43
t_410_999 = 17
t_416_999 = 19
t_431_999 = 18
t_434_999 = 56
t_442_999 = 14
t_443_999 = 23
t_444_999 = 79
t_445_999 = 8
t_446_999 = 26
t_447_999 = 19
t_448_999 = 14
t_478_999 = 12
t_483_999 = 28
Dual variables:
π[1133] = 80
π[1240] = 99
π[1241] = 

In [181]:
def PMF_n_iters(pi, sigma, n_iters, paths, path_list, flight_list, P_p, initial, current_pairs = [], n=0):
    n += 1
    print('Iteration number: ', n)
    if n > n_iters:
        print('Max number of iterations reached')
        return ":("

    tpr_prime = {}
    # tpr = (fare_p - sum (π_i) for i being each flight in path p) - bpr * (fare_r - sum (π_j) for j being each flight in path p)) - σ_p
    for p,r in P_p.keys():
        t_prime_pr = ((paths[p]['Fare'] - sum(pi[i] for i in paths[p]['Legs'])) -
                        (P_p[(p,r)]['Recapture rate']) *
                        (paths[r]['Fare'] - sum(pi[j] for j in paths[r]['Legs'])) -
                        (sigma[p]))
        if t_prime_pr < 0:
            tpr_prime[p,r] = t_prime_pr
            print(str(p)+'->'+str(r)+': ', t_prime_pr)
    #print('tpr_prime: ', tpr_prime)
    new_pairs = list(tpr_prime.keys())
    current_pairs.extend(new_pairs)

    if len(new_pairs) == 0:
        print('No new pairs, optimal solution found in previous iteration')
    
    if len(new_pairs) > 0:
        print('New pairs: ', new_pairs)
        m_n = gp.Model(str(n)+'th PMF')
        # Define the decision variables
        # t_pr: number of passengers that would like to fly on itinerary p and are reallocated to itinerary r
        t = {}
        for p in path_list:
            for r in initial:
                t[p,r] = m_n.addVar(name='t_'+str(p)+'_'+str(r),vtype = GRB.CONTINUOUS)

        # Add the new columns to the RMP
        for p,r in current_pairs:
            t[p,r] = m_n.addVar(name='t_' + str(p) +'_'+ str(r), vtype = GRB.CONTINUOUS)
        m_n.update()

        # Update the objective function
        of  = gp.quicksum((paths[p]['Fare'] - P_p[(p,r)]['Recapture rate'] * paths[r]['Fare']) * t[p,r] for r in initial for p in path_list)
        of += gp.quicksum((paths[p]['Fare'] - P_p[(p,r)]['Recapture rate'] * paths[r]['Fare']) * t[p,r] for p,r in current_pairs)
        m_n.setObjective(of, GRB.MINIMIZE)

        # Update the constraints
        m_n.addConstrs((gp.quicksum(s_ip[i, p] * t[p,r] for p in path_list for r in initial) +
                    gp.quicksum(s_ip[i, p] * t[p,r] for p, r in current_pairs) -
                    gp.quicksum(s_ip[i, r] * P_p[(p, r)]['Recapture rate'] * t[p,r] for p,r in current_pairs) >=
                    ds_i[i] for i in flight_list), name='π')

        # Constraint 2: sum t_pr <= Dp for all p
        m_n.addConstrs((t[p,r] <= paths[p]['Demand'] for p in path_list for r in initial), name='σ')
        # Constraint 3: sum t_pr >= 0 for new pairs
        m_n.addConstrs((t[p,r] <= paths[p]['Demand'] for p, r in current_pairs), name='σ')

        # Constraint 4: sum t_pr >= 0 for all p
        m_n.addConstrs((t[p,r] >= 0 for p in path_list for r in initial), name='c3')
        # Constraint 5: sum t_pr >= 0 for new pairs
        m_n.addConstrs((t[p,r] >= 0 for p, r in current_pairs), name='c3')

        # Update the model
        m_n.update()

        # Solve the model but dont show the output
        m_n.Params.OutputFlag = 0
        m_n.optimize()

        # Print the optimal objective value and the decision variables t_pr and the dual variables
        print('Optimal objective value: %g' % m_n.objVal)
        print('\nOptimal solution:')
        for v in m_n.getVars():
            if v.x > 0:
                print('%s = %g' % (v.varName, v.x))

        # print('\nDual variables:')
        # for c in m_n.getConstrs():
        #     if c.Pi != 0:
        #         print('%s = %g' % (c.ConstrName, c.Pi))

        # Save dual variables in a dictionary
        pi_new = {}
        for c in m_n.getConstrs():
            if c.constrName[0] == 'π':
                # get only the flight number from the constraint name
                flight_num_pi = int(re.findall(r'\d+', c.ConstrName)[0])    
                pi_new[flight_num_pi] = c.Pi

        sigma_new = {}
        for c in m_n.getConstrs():
            if c.constrName[0] == 'σ':
                path_num_sigma = int(re.findall(r'\d+', c.ConstrName)[0])    
                sigma_new[path_num_sigma] = c.Pi
        
        print('End of iteration number: ', n, '\n')
        PMF_n_iters(pi = pi_new,
                    sigma = sigma_new, 
                    n_iters = n_iters, 
                    paths = paths, 
                    path_list = path_list, 
                    flight_list = flight_list, 
                    P_p = P_p, 
                    initial = initial, 
                    current_pairs = current_pairs, 
                    n=n)

In [182]:
PMF_n_iters(pi_dual, sigma_dual, 50, paths=paths, path_list=path_list, flight_list=flight_list, P_p=P_p, initial = initial, current_pairs = [], n=0)


Iteration number:  1
80->58:  -2.8000000000000007
117->115:  -1.08
117->120:  -2.64
126->72:  -0.5999999999999996
128->72:  -0.7000000000000001
304->267:  -0.92
312->251:  -16.96
338->279:  -14.559999999999999
New pairs:  [(80, 58), (117, 115), (117, 120), (126, 72), (128, 72), (304, 267), (312, 251), (338, 279)]
Optimal objective value: 129138

Optimal solution:
t_9_999 = 5
t_19_999 = 26
t_48_999 = 64
t_65_999 = 9.9
t_69_999 = 2.8
t_80_999 = 14
t_112_999 = 24
t_114_999 = 4.9
t_124_999 = 23
t_125_999 = 15.1
t_126_999 = 15
t_127_999 = 12
t_129_999 = 30
t_130_999 = 18.9
t_203_999 = 37
t_227_999 = 53
t_229_999 = 12
t_230_999 = 9
t_231_999 = 24
t_232_999 = 18
t_233_999 = 5
t_241_999 = 23.52
t_265_999 = 0.37
t_266_999 = 28
t_269_999 = 8
t_271_999 = 22
t_300_999 = 28
t_301_999 = 24
t_302_999 = 12
t_325_999 = 16
t_334_999 = 27
t_341_999 = 8
t_342_999 = 5
t_346_999 = 43
t_347_999 = 14
t_350_999 = 60
t_353_999 = 10
t_357_999 = 19
t_375_999 = 37
t_380_999 = 9
t_382_999 = 20
t_390_999 = 19
t_393_

# Testing Code

In [158]:
# Create the pricing problem (PP)

tpr_prime = {}
# tpr = (fare_p - sum (π_i) for i being each flight in path p) - bpr * (fare_r - sum (π_j) for j being each flight in path p)) - σ_p
for p,r in P_p.keys():
    t_prime_pr = ((paths[p]['Fare'] - sum(pi_dual[i] for i in paths[p]['Legs'])) -
                      (P_p[(p,r)]['Recapture rate']) *
                      (paths[r]['Fare'] - sum(pi_dual[j] for j in paths[r]['Legs'])) -
                      (sigma_dual[p]))
    if t_prime_pr < 0:
        tpr_prime[p,r] = t_prime_pr

new_pairs = list(tpr_prime.keys())

# Add the new columns to the model
new_p = []
for p,r in new_pairs:
    new_p.append(p)

In [159]:
m2 = gp.Model('2nd PMF')

# Define the decision variables
# t_pr: number of passengers that would like to fly on itinerary p and are reallocated to itinerary r
t = {}
for p in path_list:
    for r in initial:
        t[p,r] = m2.addVar(name='t_'+str(p)+'_'+str(r))

# Add the new columns to the RMP
for p,r in new_pairs:
    t[p,r] = m2.addVar(name='t_' + str(p) +'_'+ str(r), lb =0)
m2.update()

# Update the objective function
of  = gp.quicksum((paths[p]['Fare'] - P_p[(p,r)]['Recapture rate'] * paths[r]['Fare']) * t[p,r] for r in initial for p in path_list)
of += gp.quicksum((paths[p]['Fare'] - P_p[(p,r)]['Recapture rate'] * paths[r]['Fare']) * t[p,r] for p,r in new_pairs)
m2.setObjective(of, GRB.MINIMIZE)

# Update the constraints
m2.addConstrs((gp.quicksum(s_ip[i, p] * t[p,r] for p in path_list for r in initial) +
              gp.quicksum(s_ip[i, p] * t[p,r] for p, r in new_pairs) -
              gp.quicksum(s_ip[i, r] * P_p[(p, r)]['Recapture rate'] * t[p,r] for p,r in new_pairs) >=
               ds_i[i] for i in flight_list), name='π')

# Constraint 2: sum t_pr <= Dp for all p
m2.addConstrs((t[p,r] <= paths[p]['Demand'] for p in path_list for r in initial), name='σ')
# Constraint 3: sum t_pr >= 0 for new pairs
m2.addConstrs((t[p,r] <= paths[p]['Demand'] for p, r in new_pairs), name='σ')

# Update the model
m2.update()

# Solve the model
m2.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 766 rows, 558 columns and 1477 nonzeros
Model fingerprint: 0xd3020dcd
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [3e+01, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 745 rows and 434 columns
Presolve time: 0.01s
Presolved: 21 rows, 124 columns, 138 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.2965440e+04   1.589875e+02   0.000000e+00      0s
Extra simplex iterations after uncrush: 1
      27    1.2913777e+05   0.000000e+00   0.000000e+00      0s

Solved in 27 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.291377700e+05


In [160]:
# Print the optimal objective value and the decision variables t_pr and the dual variables
print('Optimal objective value: %g' % m2.objVal)
print('Optimal solution:')
for v in m2.getVars():
    if v.x > 0:
        print('%s = %g' % (v.varName, v.x))

print('Dual variables:')
for c in m2.getConstrs():
    if c.Pi != 0:
        print('%s = %g' % (c.ConstrName, c.Pi))

# Save dual variables in a dictionary
pi_dual = {}
for c in m2.getConstrs():
    if c.constrName[0] == 'π':
        # get only the flight number from the constraint name
        flight_num_pi = int(re.findall(r'\d+', c.ConstrName)[0])    
        pi_dual[flight_num_pi] = c.Pi

sigma_dual = {}
for c in m2.getConstrs():
    if c.constrName[0] == 'σ':
        path_num_sigma = int(re.findall(r'\d+', c.ConstrName)[0])    
        sigma_dual[path_num_sigma] = c.Pi
        

Optimal objective value: 129138
Optimal solution:
t_9_999 = 5
t_19_999 = 26
t_48_999 = 64
t_65_999 = 9.9
t_69_999 = 2.8
t_80_999 = 14
t_112_999 = 24
t_114_999 = 4.9
t_124_999 = 23
t_125_999 = 15.1
t_126_999 = 15
t_127_999 = 12
t_129_999 = 30
t_130_999 = 18.9
t_203_999 = 37
t_227_999 = 53
t_229_999 = 12
t_230_999 = 9
t_231_999 = 24
t_232_999 = 18
t_233_999 = 5
t_241_999 = 23.52
t_265_999 = 0.37
t_266_999 = 28
t_269_999 = 8
t_271_999 = 22
t_300_999 = 28
t_301_999 = 24
t_302_999 = 12
t_325_999 = 16
t_334_999 = 27
t_341_999 = 8
t_342_999 = 5
t_346_999 = 43
t_347_999 = 14
t_350_999 = 60
t_353_999 = 10
t_357_999 = 19
t_375_999 = 37
t_380_999 = 9
t_382_999 = 20
t_390_999 = 19
t_393_999 = 26
t_409_999 = 43
t_410_999 = 17
t_416_999 = 19
t_431_999 = 18
t_434_999 = 56
t_442_999 = 14
t_443_999 = 23
t_444_999 = 79
t_445_999 = 8
t_446_999 = 26
t_447_999 = 19
t_448_999 = 14
t_478_999 = 12
t_483_999 = 28
t_80_58 = 14
t_117_115 = 10
t_117_120 = 10
t_126_72 = 15
t_304_267 = 19
t_312_251 = 36
t_338_279 =

In [161]:
# Constraints test
for i in flight_list:
    print('Flight number: ', i)
    for p in path_list:
        for r in initial:
            if s_ip[i,p] == 1:
                print(s_ip[i,p] * t[p,r])
    print('end sum 1')    
    for p, r in new_pairs:
        if s_ip[i,p] == 1:
            print(s_ip[i, p] * t[p,r])
    print('end sum 2')
    for p, r in new_pairs:
        if s_ip[i, r] == 1:
            print(s_ip[i, r] * P_p[(p, r)]['Recapture rate'] * t[p,r])
    print('end sum 3')
    print(ds_i[i], '\n')

Flight number:  1132
t_421_999
t_548_999
end sum 1
end sum 2
end sum 3
-125 

Flight number:  1133
t_408_999
t_409_999
t_410_999
end sum 1
end sum 2
end sum 3
60 

Flight number:  1140
t_513_999
end sum 1
end sum 2
end sum 3
-204 

Flight number:  1160
t_468_999
t_476_999
t_496_999
t_544_999
end sum 1
end sum 2
end sum 3
-90 

Flight number:  1161
t_326_999
end sum 1
end sum 2
end sum 3
-113 

Flight number:  1240
t_9_999
t_19_999
t_37_999
t_82_999
end sum 1
end sum 2
end sum 3
31 

Flight number:  1241
t_224_999
t_225_999
t_226_999
t_227_999
t_228_999
t_229_999
t_230_999
t_231_999
t_232_999
t_233_999
end sum 1
end sum 2
end sum 3
121 

Flight number:  1248
t_350_999
t_375_999
t_380_999
t_390_999
t_401_999
t_416_999
t_431_999
t_438_999
t_444_999
t_505_999
end sum 1
end sum 2
end sum 3
241 

Flight number:  1249
t_46_999
t_47_999
t_48_999
t_49_999
t_50_999
t_51_999
end sum 1
end sum 2
end sum 3
64 

Flight number:  1252
t_95_999
t_98_999
t_108_999
t_114_999
t_125_999
t_134_999
t_152_999

In [162]:
# Create the pricing problem (PP)

tpr_prime = {}
# tpr = (fare_p - sum (π_i) for i being each flight in path p) - bpr * (fare_r - sum (π_j) for j being each flight in path p)) - σ_p
for p,r in P_p.keys():
    t_prime_pr = ((paths[p]['Fare'] - sum(pi_dual[i] for i in paths[p]['Legs'])) -
                      (P_p[(p,r)]['Recapture rate']) *
                      (paths[r]['Fare'] - sum(pi_dual[j] for j in paths[r]['Legs'])) -
                      (sigma_dual[p]))
    if t_prime_pr < 0:
        tpr_prime[p,r] = t_prime_pr

new_pairs = list(tpr_prime.keys())
print(new_pairs)

# Add the new columns to the model
new_p = []
for p,r in new_pairs:
    new_p.append(p)

if len(new_p) == 0:
    print('Optimal solution found')

[(80, 58), (265, 123), (304, 267), (312, 251)]


In [163]:
# Objective function test
for r in initial:
    for p in path_list:
        print((paths[p]['Fare'] - P_p[(p,r)]['Recapture rate'] * paths[r]['Fare']) * t[p,r])

for p,r in new_pairs:
    print(paths[p]['Fare'] - P_p[(p, r)]['Recapture rate'] * paths[r]['Fare'] * t[p,r])


105.0 t_0_999
52.0 t_1_999
53.0 t_2_999
104.0 t_3_999
54.0 t_4_999
53.0 t_5_999
45.0 t_6_999
97.0 t_7_999
90.0 t_8_999
99.0 t_9_999
104.0 t_10_999
109.0 t_11_999
50.0 t_12_999
52.0 t_13_999
48.0 t_14_999
95.0 t_15_999
106.0 t_16_999
99.0 t_17_999
100.0 t_18_999
96.0 t_19_999
95.0 t_20_999
92.0 t_21_999
99.0 t_22_999
102.0 t_23_999
94.0 t_24_999
45.0 t_25_999
50.0 t_26_999
53.0 t_27_999
85.0 t_28_999
45.0 t_29_999
54.0 t_30_999
103.0 t_31_999
91.0 t_32_999
99.0 t_33_999
94.0 t_34_999
95.0 t_35_999
91.0 t_36_999
105.0 t_37_999
92.0 t_38_999
100.0 t_39_999
99.0 t_40_999
45.0 t_41_999
50.0 t_42_999
52.0 t_43_999
50.0 t_44_999
48.0 t_45_999
106.0 t_46_999
45.0 t_47_999
127.0 t_48_999
47.0 t_49_999
51.0 t_50_999
47.0 t_51_999
46.0 t_52_999
91.0 t_53_999
97.0 t_54_999
98.0 t_55_999
100.0 t_56_999
92.0 t_57_999
106.0 t_58_999
67.0 t_59_999
67.0 t_60_999
98.0 t_61_999
104.0 t_62_999
94.0 t_63_999
93.0 t_64_999
98.0 t_65_999
105.0 t_66_999
53.0 t_67_999
138.0 t_68_999
90.0 t_69_999
107.0 t_70_99

KeyError: (265, 123)